In [151]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

In [10]:
from google.colab import files
files.upload()

{}

In [11]:
dataset = pd.read_json("intents.json")

In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [39]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [61]:
words = []
labels = []
documents_x = []
documents = []
documents_y = []

for intent in dataset['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        wrds = [ps.stem(w.lower()) for w in wrds if w != "?"]
        words.extend(wrds)
        documents.append((wrds, intent['tag']))
        documents_x.append(' '.join(wrds))
        documents_y.append(intent['tag'])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [62]:
documents_x 

['hi',
 'how are you',
 'is anyon there',
 'hello',
 'good day',
 'what up',
 'cya',
 'see you later',
 'goodby',
 'i am leav',
 'have a good day',
 'how old',
 'how old is tim',
 'what is your age',
 'how old are you',
 'age',
 'what is your name',
 'what should i call you',
 'what your name',
 'id like to buy someth',
 'what on the menu',
 'what do you reccommend',
 'could i get someth to eat',
 'when are you guy open',
 'what are your hour',
 'hour of oper']

In [43]:
print(len(documents), "documents")
print(len(labels), "documents", labels)
print(len(words), "unique steemed words", words)

26 documents
6 documents ['greeting', 'goodbye', 'age', 'name', 'shop', 'hours']
82 unique steemed words ['hi', 'how', 'are', 'you', 'is', 'anyon', 'there', 'hello', 'good', 'day', 'what', 'up', 'cya', 'see', 'you', 'later', 'goodby', 'i', 'am', 'leav', 'have', 'a', 'good', 'day', 'how', 'old', 'how', 'old', 'is', 'tim', 'what', 'is', 'your', 'age', 'how', 'old', 'are', 'you', 'age', 'what', 'is', 'your', 'name', 'what', 'should', 'i', 'call', 'you', 'what', 'your', 'name', 'id', 'like', 'to', 'buy', 'someth', 'what', 'on', 'the', 'menu', 'what', 'do', 'you', 'reccommend', 'could', 'i', 'get', 'someth', 'to', 'eat', 'when', 'are', 'you', 'guy', 'open', 'what', 'are', 'your', 'hour', 'hour', 'of', 'oper']


In [45]:
training =[]
output = []

output_empty = [0] * len(labels)
output_empty

[0, 0, 0, 0, 0, 0]

In [127]:
from sklearn.feature_extraction.text import CountVectorizer
cv_x = CountVectorizer(max_features=45)
train_x = cv_x.fit_transform(documents_x).toarray()
cv_y = CountVectorizer(max_features=6)
train_y = cv_y.fit_transform(documents_y).toarray()

In [113]:
print(len(cv_x.get_feature_names()))

45


In [110]:
print(train_x.shape)
print(train_y.shape)

(26, 45)
(26, 6)


In [89]:
!pip install tflearn 

     |████████████████████████████████| 112kB 6.8MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-cp37-none-any.whl size=127300 sha256=49b2acb30b74a0840f50a1177f4a9fc68b2a0f80d7fe30fed329b7d882c0a9e3
  Stored in directory: /root/.cache/pip/wheels/31/d2/ed/fb9a0d301dd9586c11e9547120278e624227f22fd5f4baf744
Successfully built tflearn


In [90]:
import tensorflow as tf
import tflearn

Instructions for updating:
non-resource variables are not supported in the long term


In [91]:
tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [92]:
model = tflearn.DNN(net, tensorboard_dir="tflearn_logs")

model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.02119 | time: 0.018s
| Adam | epoch: 1000 | loss: 0.02119 - acc: 1.0000 -- iter: 24/26
Training Step: 4000  | total loss: 0.01947 | time: 0.026s
| Adam | epoch: 1000 | loss: 0.01947 - acc: 1.0000 -- iter: 26/26
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [135]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [ps.stem(w.lower()) for w in sentence_words]
  return [' '.join(sentence_words)]

In [146]:
def bag_of_words(sentence):
  sentence_words = clean_up_sentence(sentence)
  test = sentence_words + documents_x
  bag = cv_x.fit_transform(test).toarray()
  return(bag[0])

In [148]:
print(bag_of_words('Hi'))
results = model.predict([bag_of_words('Hi')])
results_index = np.argmax(results)
results_index

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0]


2

In [152]:
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp)])
        results_index = np.argmax(results)
        tag = labels[results_index]

        for tg in dataset["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: hi
18 years young!
You: how are you
I am 18 years old!
You: Goodbye
Sad to see you go :(
You: your name
We sell chocolate chip cookies for $2!
You: quit
